In [10]:
import os
import glob
import pandas as pd
from datetime import datetime, timedelta

In [2]:

MAIN_BITS_DICT = {
    "version_number":3,
    "pkt_type":1,
    "sec_hdr_flag":1,
    "apid":11,
    "seq_flags":2,
    "seq_count":14,
    "pkt_data_length":16,
    "secondary_header": None,
    "data":None,
    "checksum":16
}

def hex_to_binary(hex_string: str) -> str:
    """
    Function to convert hex string to a binary string with 4-bit representation
    Converts each hex char to integer, then format as 4-bit binary and adds it to
    a final binary string
    """
    binary_string = ""
    for char in hex_string:
        binary_string += format(int(char, 16), '04b')
    return binary_string

def read_through_binary_str_and_update_pointer(binary_string: str, pointer: int, number_of_bits: int) -> tuple[str, int]:
    """Function to read an specific amount of bits from the binary string given the 
    starting point at the pointer vector. Updates the pointer to end of the bits read and returns the content
    read and the new pointer.
    """
    component_binary = binary_string[pointer:(pointer+number_of_bits)]
    new_pointer = pointer+number_of_bits

    return component_binary, new_pointer 

def adjust_bit_size_for_variable_components(component: str, number_of_bits_dict: dict, space_packet_dict: dict) -> int:
    """Function to adjust the bit size for variable components. Returns the bit size for the component given as input.
    """
    if component == "secondary_header":
        try:
            bit_size = 64 if space_packet_dict['sec_hdr_flag'] == '1' else 0
        except KeyError:
            raise KeyError("Attempted to access 'sec_hdr_flag' before it being available!")
    elif component == "data":
        try:
            pkt_data_field_bits = (int(space_packet_dict['pkt_data_length'], 2) + 1)*8
            checksum_bits = number_of_bits_dict["checksum"]
            secondary_header_bits = number_of_bits_dict["secondary_header"]
            
            bit_size = pkt_data_field_bits - secondary_header_bits - checksum_bits
        except KeyError:
            raise KeyError("Attempted to access 'pkt_data_length' or 'secondary_header' before it being available!")
    else:
        bit_size = number_of_bits_dict[component]
    
    return bit_size

def read_binary_str_to_space_packet(binary_string: str) -> tuple[dict, int]:
    """Main function to read the hex string. It will turn the hex into binary, then iterate over the components of
    the space packet and dinamically adjusts the bit size. Returns the space packet in a dict format with the components
    as binary strings.
    """

    number_of_bits_dict = MAIN_BITS_DICT.copy()
    
    space_packet_dict = dict()
    pointer = 0
    for component in number_of_bits_dict.keys():
        number_of_bits_dict[component] = adjust_bit_size_for_variable_components(component, number_of_bits_dict, space_packet_dict)
        space_packet_dict[component], pointer = read_through_binary_str_and_update_pointer(binary_string, pointer, number_of_bits_dict[component])

    space_packet_bit_size = sum(number_of_bits_dict.values())

    return space_packet_dict, space_packet_bit_size

def read_through_hex_str(hex_string: str) -> list[dict]:
    binary_string = hex_to_binary(hex_string)

    space_packet_list = []

    space_packet_bit_size = 0
    while space_packet_bit_size <= len(binary_string):
        space_packet, space_packet_bit_size = read_binary_str_to_space_packet(binary_string)
        binary_string = binary_string[space_packet_bit_size:]
        space_packet_list.append(space_packet)

    return space_packet_list

# hex_string = "0808c3f3002d000008e911e9cebe070007e805d10a5403bb02530a060955097f02c10da705d20a0703ba025109ac09570981e777"
# read_through_hex_str(hex_string)

In [3]:


def convert_64bit_binary_to_datetime(binary_string: str) -> datetime| pd._libs.tslibs.nattype.NaTType:
    """
    GPS time is composed of a 32-bit week field and a 32-bit ms field. This takes the week and milliseconds
    and transform them to datetime object.
    """
    if binary_string == '':
        return pd.NaT
    elif len(binary_string) != 64:
        raise ValueError("The binary string must be exactly 64 bits long")

    week_bits = binary_string[:32]
    ms_bits = binary_string[32:]

    week = int(week_bits, 2)
    ms = int(ms_bits, 2)

    return gps_time_to_datetime(week, ms)

def gps_time_to_datetime(week, ms):
    """The week field indicates the unsigned integer number of weeks elapsed since the beginning of the current GPS epoch (which
    started on January 6, 1980). The ms field indicates the unsigned integer number of milliseconds
    elapsed since the beginning of the current week. Returns the datetime formatted date.
    """
    gps_epoch = datetime(1980, 1, 6)

    days = week * 7
    total_timedelta = timedelta(days=days, milliseconds=ms)

    result_datetime = gps_epoch + total_timedelta

    return result_datetime


def create_df_from_space_packets(packets: list[dict]) -> pd.DataFrame:

    df = pd.DataFrame(packets)

    df['version_number'] = df['version_number'].apply(lambda x: int(x, 2))
    df['apid'] = df['apid'].apply(lambda x: hex(int(x, 2)))
    df['seq_flags'] = df['seq_flags'].apply(lambda x: hex(int(x, 2)))
    df['pkt_data_length'] = df['pkt_data_length'].apply(lambda x: hex(int(x, 2)))
    df['secondary_header'] = df['secondary_header'].apply(convert_64bit_binary_to_datetime)

    return df


In [4]:

file_path = 'decoded_satcs_dump\\hk_eps_only.out'

def read_hex_file_to_hex_str(file_path: str) -> str:
    with open(file_path, 'r') as f:
        all_lines = ''.join(line.strip() for line in f)
    return all_lines

def binary_file_to_hex(file_path: str) -> str:
    with open(file_path, 'rb') as file:
        binary_data = file.read()
    hex_data = binary_data.hex()
    return hex_data

def read_file_and_get_space_packets(file_path: str) -> list[dict]:
    try:
        hex_data = read_hex_file_to_hex_str(file_path)
    except UnicodeDecodeError:
        hex_data = binary_file_to_hex(file_path)
    
    packets = read_through_hex_str(hex_data)

    return packets


In [5]:
#file_path = 'decoded_satcs_dump\\hk_vur_inst_only.out'
file_path = 'decoded_satcs_dump\\adcs_housekeeping_only.out'
#file_path = 'decoded_satcs_dump\\hk_eps_only.out'

space_packets = read_file_and_get_space_packets(file_path)
df = create_df_from_space_packets(space_packets)

df

,version_number,pkt_type,sec_hdr_flag,apid,seq_flags,seq_count,pkt_data_length,secondary_header,data,checksum
0,0,0,1,0x14,0x3,00000000000000,0x6d,2023-09-27 11:28:49.968,0000000111001111100000110001111100000000000000...,1000110101101011
1,0,0,1,0x32,0x3,00000000000001,0x8f,2023-09-27 11:28:49.968,0000000000000000000000000000000000000000000000...,1100011111001110
2,0,0,0,0x15,0x1,00000000000000,0xe4,NaT,0000000000000000000010001110100100010001111010...,0010100110111000
3,0,0,0,0x15,0x2,00000000000001,0x39,NaT,0000000000000000000000000000000000000000000000...,1000010110101000
4,0,0,0,0x33,0x1,00000000000000,0xe4,NaT,0000000000000000000010001110100100010001111010...,0100111011001010
...,...,...,...,...,...,...,...,...,...,...
324,0,0,1,0x14,0x3,00000001111100,0x6d,2023-09-27 11:33:59.970,0000000111010100001111100000111100000000000000...,1110000110000011
325,0,0,1,0x32,0x3,00000001111101,0x8f,2023-09-27 11:33:59.970,0000000000000000000000000000000000000000000000...,0101001000100010
326,0,0,0,0x15,0x1,00000000000000,0xe4,NaT,0000000000000000000010001110100100010001111011...,1101100101110010
327,0,0,0,0x15,0x2,00000000000001,0x39,NaT,0000000000000000000000000000000000000000000000...,0100111100000101


In [6]:
folder_path = 'decoded_satcs_dump' 
file_pattern = os.path.join(folder_path, '*')  

files = glob.glob(file_pattern)

all_packets = []
for file_path in files:
    space_packets = read_file_and_get_space_packets(file_path)  
    all_packets.extend(space_packets)

all_packets

[{'version_number': '000',
  'pkt_type': '0',
  'sec_hdr_flag': '1',
  'apid': '00000010100',
  'seq_flags': '11',
  'seq_count': '00000000000000',
  'pkt_data_length': '0000000001101101',
  'secondary_header': '0000000000000000000010001110100100010001111010011011100100110000',
  'data': '00000001110011111000001100011111000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000010000000000000000000000000000000000000000000000

In [7]:
def transform_adc_value_from_binary(binary_input):
    # Ensure the binary string is 16 bits long
    if len(binary_input) != 16:
        raise ValueError("The binary input must be exactly 16 bits long")

    # Convert the binary string to an integer
    input_16bit = int(binary_input, 2)

    # Extract the 12-bit ADC value from the 16-bit input (lower 12 bits)
    adc_value = input_16bit & 0x0FFF  # Mask to get the lower 12 bits

    # Apply the transformation formula
    transformed_value = adc_value * 13.352 - 22300

    return transformed_value

#transform_adc_value_from_binary('0000100101111001')

In [8]:
def transform_adc_value(adc_value, conversion_formula):
    """
    Apply the conversion formula to the ADC value.
    """
    return eval(conversion_formula)

def process_binary_string(binary_string):

    if len(binary_string) % 16 != 0:
        raise ValueError("The binary input length must be a multiple of 16 bits")

    # Conversion formulas from the provided image
    conversion_formulas = [
        "adc*13.352 - 22300",           # Instantaneous Doppler offset
        "adc*0.03 - 152",               # Instantaneous RSSI
        "adc*0.00488",                # Power bus voltage
        "adc*0.16643964",             # Total supply current
        "adc*0.16643964",             # Transmitter current
        "adc*0.16643964",             # Receiver current
        "adc*0.16643964",             # Power amplifier current
        "adc*(-0.07669) + 195.6037",       # Power amplifier temperature
        "adc*(-0.07669) + 195.6037",       # Local oscillator temperature
        "adc*adc*5.887*10**-5",           # Instantaneous RF reflected power
        "adc*adc*5.887*10**-5",           # Instantaneous RF forward power
        "adc*0.00488",                # Power bus voltage (repeated)
        "adc*0.16643964",             # Total supply current (repeated)
        "adc*0.16643964",             # Transmitter current (repeated)
        "adc*0.16643964",             # Receiver current (repeated)
        "adc*0.16643964",             # Power amplifier current (repeated)
        "adc*(-0.07669) + 195.6037",       # Power amplifier temperature (repeated)
        "adc*(-0.07669) + 195.6037"        # Local oscillator temperature (repeated)
    ]

    # Split the binary string into 16-bit chunks
    num_values = len(binary_string) // 16
    results = []

    for i in range(num_values):
        # Extract 16-bit segment
        segment = binary_string[i*16:(i+1)*16]

        # Convert binary segment to integer
        input_16bit = int(segment, 2)

        # Extract the 12-bit ADC value from the 16-bit input (lower 12 bits)
        adc_value = input_16bit & 0x0FFF  # Mask to get the lower 12 bits

        # Apply the corresponding conversion formula
        if i < len(conversion_formulas):
            formula = conversion_formulas[i].replace("adc", str(adc_value))
            transformed_value = transform_adc_value(adc_value, formula)
            results.append(transformed_value)
        else:
            results.append(None) 

    return results

file_path = 'decoded_satcs_dump\\hk_vur_inst_only.out'

space_packets = read_file_and_get_space_packets(file_path)
df = create_df_from_space_packets(space_packets)


transformed_values = process_binary_string(df['data'][0])
for i, value in enumerate(transformed_values):
    print(f'Transformed value {i+1}: {value}')


Transformed value 1: 10078.600000000002
Transformed value 2: -82.19
Transformed value 3: 8.07152
Transformed value 4: 49.59901272
Transformed value 5: 12.64941264
Transformed value 6: 98.03294796
Transformed value 7: 0.16643964
Transformed value 8: 10.62742000000003
Transformed value 9: 8.63348000000002
Transformed value 10: 29.34292732
Transformed value 11: 717.86454768
Transformed value 12: 7.88608
Transformed value 13: 402.95036844
Transformed value 14: 160.28137332
Transformed value 15: 98.36582724
Transformed value 16: 418.09637568
Transformed value 17: 10.550730000000016
Transformed value 18: 8.480100000000022


In [9]:
df['data'].apply(process_binary_string)

0     [10078.600000000002, -82.19, 8.07152, 49.59901...
1     [8302.784, -79.76, 7.373679999999999, 53.59356...
2     [8916.976000000002, -84.68, 8.05688, 49.266133...
3     [4671.040000000001, -89.63, 7.93, 204.22143828...
4     [1626.7839999999997, -91.28, 7.266319999999999...
5     [10385.696, -82.64, 7.89584, 401.45241168, 158...
6     [7701.9439999999995, -84.77, 7.90072, 406.7784...
7     [7101.103999999999, -80.0, 7.2565599999999995,...
8     [8022.392, -78.59, 7.98368, 384.97488732, 159....
9     [5819.312000000002, -82.94, 8.1252, 376.819344...
10    [9237.423999999999, -86.39, 7.9056, 392.963990...
11    [9744.8, -91.73, 7.91048, 400.2873342, 158.783...
12    [11413.800000000003, -88.1, 7.93488, 396.29278...
13    [8957.032, -86.48, 7.95928, 386.1399648, 158.4...
Name: data, dtype: object